
<div class="alert alert-block alert-info">
<h1>Data set-up and preprocessing</h1>

In [6]:
import pandas as pd
import warnings                  
             
warnings.filterwarnings('ignore')

# [TODO] you need to change this path to your local data path
data_path = './data/train.csv'

df = pd.read_csv('./data/train.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values(by='Datetime').rename(columns={'Datetime' : 'ds', 'ActivePower' : 'y'})

# [TODO] We would use different method to aggregate features (X)
df = df.set_index('ds').resample('D').mean().reset_index()

last_date = df['ds'].iloc[-1]

<div class="alert alert-block alert-warning">  
<h3> [Demo] SARIMAX model (Inseong Han)</h3>

In [4]:
from helper import parameter_mixer
from cross_validation import BaseTimeSeriesModelCrossValidation
from sarimax_model import SARIMAX

model = SARIMAX(m=7)
p_values = [0, 1]
d_values = [0]
q_values = [0, 1]
P_values = [1, 2]
D_values = [1]
Q_values = [1, 2]
t_values = ['n']


params = parameter_mixer(
    ['p', 'd', 'q', 'P', 'D', 'Q', 't'],
    [p_values, d_values, q_values, P_values, D_values, Q_values, t_values]
)

cv = BaseTimeSeriesModelCrossValidation(num_fold=5, rolling_size=10, validation_size=15)
training_history, best_p = cv.cross_validation(df['y'], model, params)

model.reset()
model.fit(df['y'], best_p)
model

[SARIMAX] Start cross-validation


  6%|▋         | 1/16 [00:01<00:23,  1.58s/it]

[SARIMAX] P={'p': 0, 'd': 0, 'q': 0, 'P': 1, 'D': 1, 'Q': 1, 't': 'n'} RMSE=242.383


 12%|█▎        | 2/16 [00:06<00:46,  3.33s/it]

[SARIMAX] P={'p': 0, 'd': 0, 'q': 0, 'P': 1, 'D': 1, 'Q': 2, 't': 'n'} RMSE=242.294


...

[SARIMAX] P={'p': 0, 'd': 0, 'q': 0, 'P': 2, 'D': 1, 'Q': 1, 't': 'n'} RMSE=242.122


 31%|███▏      | 5/16 [00:18<00:42,  3.82s/it]

[SARIMAX] P={'p': 0, 'd': 0, 'q': 1, 'P': 1, 'D': 1, 'Q': 1, 't': 'n'} RMSE=233.911


100%|██████████| 16/16 [01:46<00:00,  6.66s/it]


[SARIMAX] P={'p': 0, 'd': 0, 'q': 1, 'P': 1, 'D': 1, 'Q': 1, 't': 'n'} Best RMSE=233.911


<div class="alert alert-block alert-warning">  
<h3> [Submission]</h3>

In [26]:
from datetime import timedelta
submission = model.forecast(15).reset_index(drop=True)
submission.index = last_date + pd.to_timedelta(submission.index + 1, unit='D')
submission.index.name = 'Date'
submission.name = 'Forecasting'
submission.to_csv('submission.csv', index=True)